In [17]:

import numpy as np
from scipy.integrate import solve_bvp
from scipy.integrate._bvp import BVPResult
from matplotlib import pyplot as plt

from typing import List, Tuple, Callable

from matplotlib import animation
import matplotlib as mpl

from collections import defaultdict

import subprocess
import shlex

In [18]:

# ----- WARNING SUPPRESSION -----
# This should not be used lightly. This is primarily for formatting reasons for our paper.
import warnings
warnings.filterwarnings('ignore')

DEFAULT_FIGSIZE = (5, 5)
plt.rcParams['figure.figsize'] = DEFAULT_FIGSIZE
plt.rcParams.update({'font.size': 6})

mpl.rcParams["figure.dpi"] = 300

# ---- ENABLE LATEX MATPLOTLIB FONT -----
# Don't run this cell if you have issues with your latex installation
# Use LaTeX fonts
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Computer Modern Roman"
})

plt.style.use("seaborn-v0_8-muted")

In [19]:

PlanetType = Tuple[float, Callable[[np.ndarray], np.ndarray], Callable[[np.ndarray], np.ndarray]]

# this stores a list of planets with each planet's mass and position with [mass, pos_x, pos_y]

def norm(sx, sy, px, py):
        return ((sx-px)**2 + (sy-py)**2)**(1/2)

def best_path(
    planets: List[PlanetType],
    T: float,
    figname: str | None = None,
    animname: str | None = None,
    N_COMPUTE_STEPS: int = 10000,
    N_ANIM_FRAMES: int = 50,
    ANIM_LEN_SEC: float = 3,
    max_nodes: int = 100000,
) -> BVPResult:
    """Solve a boundary value problem to get from planet 0 to planet 1. Graph and animate results.

    Args:
        planets (List[PlanetType]): List of planets. Must have at least 2.
        T (float): Final time.
        figname (str | None, optional): If str, save a figure to this path. Defaults to None.
        animname (str | None, optional): If str, save an animation to this path. Defaults to None.
        N_COMPUTE_STEPS (int, optional): Number of time steps to use. Defaults to 10000.
        N_ANIM_FRAMES (int, optional): Number of frames in the animation total. Defaults to 50.
        ANIM_LEN_SEC (float, optional): Duration of the animation. Defaults to 3.
        max_nodes (int, optional): Max nodes for BVP solver. 1000 ends too quickly.
    Returns:
        BVPResult: Result of solving the BVP.
    """
    
    #G = 6.674e-11
    G=0.1
    def ode(t, y):
        '''
        sx: spaceship x position
        sy: spaceship y position
        dsx: spaceship x velocity
        dsy: spaceship y velocity
        p1, p2, p3, p4: costate vars
            '''
        sx, sy = y[0], y[1]
        dsx, dsy = y[2], y[3]
        p1, p2, p3, p4 = y[4], y[5], y[6], y[7]

        ddsx = G * sum([mp*(sx-px(t))/(norm(sx, sy, px(t), py(t)))**3 for mp, px, py in planets] + p3/2)
        ddsy = G * sum([mp*(sy-py(t))/(norm(sx, sy, px(t), py(t)))**3 for mp, px, py in planets] + p4/2)

        dp1 = -p3*(G * sum([mp/norm(sx, sy, px(t), py(t))**3 - 3*mp*(sx - px(t))**2/norm(sx, sy, px(t), py(t))**5 for mp, px, py in planets]))
        dp2 = -p4*(G * sum([mp/norm(sx, sy, px(t), py(t))**3 - 3*mp*(sy - py(t))**2/norm(sx, sy, px(t), py(t))**5 for mp, px, py in planets]))

        dp3 = -p1
        dp4 = -p2

        return np.array([dsx, dsy, ddsx, ddsy, dp1, dp2, dp3, dp4])

    target_start_x = planets[0][1](0)
    target_start_y = planets[0][2](0)
    target_end_x = planets[1][1](T)
    target_end_y = planets[1][2](T)
    def bc(ya, yb):
        return np.array([
            # Start at first planet x, y, with no velocity
            ya[0] - target_start_x,
            ya[1] - target_start_y,
            ya[2] - 0,
            ya[3] - 0,
            
            # End at second planet's x/y, with no velocity
            yb[0] - target_end_x,
            yb[1] - target_end_y,
            yb[2] - 0,
            yb[3] - 0,
        ])

    t = np.linspace(0, T, N_COMPUTE_STEPS)
    y_guess = np.zeros((8, t.size))
    # y_guess[0:2,0] = [ship_start_x, ship_start_y]
    # y_guess[0:2,-1] = [ship_end_x, ship_end_y]

    print("Running solve_bvp...")
    soln = solve_bvp(ode, bc, t, y_guess, verbose=2, max_nodes=max_nodes)

    sx = soln.sol(t)[0]
    sy = soln.sol(t)[1]

    ux = soln.sol(t)[6]/2
    uy = soln.sol(t)[7]/2

    fig, (ax1, ax2) = plt.subplots(2,1, gridspec_kw={"height_ratios": [2, 1]})
       
    masses = [p[0] for p in planets]
    min_mass = min(masses)
    max_mass = max(masses)
    min_radius = 5
    max_radius = 12
    def mass_to_radius(mass):
        return min_radius + (mass - min_mass) / (max_mass - min_mass+0.1) * (max_radius - min_radius)
    
    planet_x_ts = []
    planet_y_ts = []
    planet_trajectories = []
    planet_points = []
    for i, (pm, px, py) in enumerate(planets):
        pxt = px(t)
        pyt = py(t)
        planet_x_ts.append(pxt)
        planet_y_ts.append(pyt)
        color = next(ax1._get_lines.prop_cycler)['color']
        planet_trajectories.append(ax1.plot(pxt, pyt, label=f'planet {i+1}', color=color)[0])
        
        
        
        planet_points.append(ax1.plot(pxt[-1], pyt[-1], "o", color=color, markersize=mass_to_radius(pm))[0])
    
    control_x_graph, = ax2.plot(t, ux, label='control in x direction')
    control_y_graph, = ax2.plot(t, uy, label='control in y direction')

    color = next(ax1._get_lines.prop_cycler)['color']

    ss_point, = ax1.plot(sx[-1], sy[-1], "o", color=color)
    ss_trajectory_plot, = ax1.plot(sx, sy, color=color, label='Optimal Path')
    
    ax1.set(xlabel="x", ylabel="y", title="Optimal Path")
    ax1.legend(loc="center left", bbox_to_anchor=(1, 0.5, 0.3, 0.3), bbox_transform=ax1.transAxes)
    ax2.set(title='Optimal Control', xlabel="Time", ylabel="Acceleration")
    ax2.legend(loc="center left", bbox_to_anchor=(1, 0.5, 0.3, 0.3), bbox_transform=ax2.transAxes)
    fig.tight_layout()
    fig.show()
    # Save Figure if applicable
    if figname:
        fig.savefig(figname, dpi=300)
        print(f"Figure saved to {figname}")


    # Animation
    def update_anim(anim_frame):
        
        i = min(N_COMPUTE_STEPS-1, int((anim_frame+1) / N_ANIM_FRAMES * N_COMPUTE_STEPS))
        
        # update spaceship graph
        ss_trajectory_plot.set_xdata(sx[:i+1])
        ss_trajectory_plot.set_ydata(sy[:i+1])
        # update spaceship dot
        ss_point.set_xdata(sx[i])
        ss_point.set_ydata(sy[i])
        
        # update planets
        for j, _ in enumerate(planets):
            planet_trajectories[j].set_xdata(planet_x_ts[j][:i+1])
            planet_trajectories[j].set_ydata(planet_y_ts[j][:i+1])
            
            planet_points[j].set_xdata(planet_x_ts[j][i])
            planet_points[j].set_ydata(planet_y_ts[j][i])
        
        # update control expenditure
        control_x_graph.set_xdata(t[:i+1])
        control_x_graph.set_ydata(ux[:i+1])
        
        control_y_graph.set_xdata(t[:i+1])
        control_y_graph.set_ydata(uy[:i+1])
        
    # Run and Save Animation if applicable
    if animname: 
        print("Saving animation...")   
        anim = animation.FuncAnimation(fig, update_anim, range(N_ANIM_FRAMES), interval=ANIM_LEN_SEC * 1000 // N_ANIM_FRAMES)
        anim.save(animname)
        print(f"Animation saved to {animname}")

    plt.close()
    
    

We saw that with T = 100000 we got a cool graphic

In [20]:
best_path(
    [
        (5, lambda t:np.ones_like(t)*-1, lambda t:np.ones_like(t)*-1),
        (10, lambda t: np.ones_like(t)*10, lambda t: np.ones_like(t)*10),
        (10, lambda t:np.ones_like(t)*0, lambda t:np.ones_like(t)*5)
    ],
    10, 
    figname='fig1.pdf',
    animname="anim1.mp4",
    max_nodes=50000
)

Running solve_bvp...
   Iteration    Max residual  Max BC residual  Total nodes    Nodes added  
       1          1.33e+01       2.46e+00         10000          19998     


KeyboardInterrupt: 

In [ ]:
best_path(
    [
        (10, lambda t:3*np.cos(t)+1, lambda t:np.sin(t)-2),
        (20, lambda t:2*np.cos(t+np.pi)-3, lambda t:.5*np.sin(t+np.pi)+4),
        (15, lambda t:np.zeros_like(t)-2, lambda t:np.zeros_like(t)+1),
    ], 
    np.pi*2,
    figname="fig2.pdf",
    animname="anim2.mp4",
    max_nodes=300000
)

Running solve_bvp...
   Iteration    Max residual  Max BC residual  Total nodes    Nodes added  
       1          2.31e+02       9.57e-01         10000          19998     
       2          1.08e+02       4.81e-01         29998          59994     
       3          1.26e+02       2.38e-01         89992         179982     
       4          9.86e+02       1.37e-01        269974        (539946)    
Number of nodes is exceeded after iteration 4. 
Maximum relative residual: 9.86e+02 
Maximum boundary residual: 1.37e-01
Figure saved to fig2.pdf
Saving animation...
Animation saved to anim2.mp4


In [ ]:
best_path(
    [
        (1, lambda t:3*np.cos(t)+1, lambda t:np.sin(t)-4),
        (2, lambda t:2*np.cos(t+np.pi)-2, lambda t:.5*np.sin(t+np.pi)+7),
        (200, lambda t:np.zeros_like(t)-1, lambda t:np.zeros_like(t)+4),
    ], 
    np.pi*2,
    figname="fig3.pdf",
    animname="anim3.mp4",
)

Running solve_bvp...
   Iteration    Max residual  Max BC residual  Total nodes    Nodes added  
       1          7.96e+01       1.72e+00         10000          19998     
       2          2.22e+03       3.03e-01         29998          59994     
       3          1.33e+03       4.26e-02         89992        (177935)    
Number of nodes is exceeded after iteration 3. 
Maximum relative residual: 1.33e+03 
Maximum boundary residual: 4.26e-02
Figure saved to fig3.pdf
Saving animation...
Animation saved to anim3.mp4


In [24]:
best_path(
    [
        (1, lambda t:10*np.cos(t)+10, lambda t:5*np.sin(t)-4),
        (10, lambda t:-7*np.cos(3*t+np.pi)-4, lambda t:.5*np.sin(0.1*t+np.pi)+7),
        (10, lambda t:-7*np.cos(4*t+np.pi)+5, lambda t:4*np.sin(2*t+np.pi)+2),
        (20, lambda t:4*np.zeros_like(1+t)-1, lambda t:np.zeros_like(t)+4),
    ], 
    np.pi*2,
    figname="fig4.pdf",
    animname="anim4.mp4",
    max_nodes=200000
)

Running solve_bvp...
   Iteration    Max residual  Max BC residual  Total nodes    Nodes added  
       1          9.91e+02       2.34e+00         10000          14733     
       2          1.46e+03       5.77e-01         24733          31490     
       3          2.69e+05       1.22e-01         56223          83849     
       4          2.11e+06       7.02e-02        140072        (233643)    
Number of nodes is exceeded after iteration 4. 
Maximum relative residual: 2.11e+06 
Maximum boundary residual: 7.02e-02
Figure saved to fig4.pdf
Saving animation...
Animation saved to anim4.mp4


In [26]:
def combine_test_videos(output_video_name, anim_filenames):
    """Create a unified video that combines the test videos.
    
    This function suppresses errors during running ffmpeg, so
    if it's not working, make sure you have ffmpeg installed on 
    your computer.
    """
    print(f"Combining into {anim_filenames}")
    files = anim_filenames
    
    # Grab singular test video files in this directory.
    # for f in glob.glob("test_*.mp4", root_dir="."):
    #     files.append(f)
    
    # Create a grid by getting the n. rows needed, and duplicating the last video until it's a rectangle
    n_rows = len(anim_filenames) // 3
    n_leftover_spots = len(anim_filenames) % 3
    files += [files[-1] for _ in range(n_leftover_spots)]
    i = 0
    
    # at this point, there are enough videos in files to be rectangular.
    n_rows = len(files) // 3
    
    # Now we construct an ffmpeg command to combine the videos into a 3-column grid.
    inclusion_rows = " ".join([f"-i {files[3*r]} -i {files[3*r+1]} -i {files[3*r+2]}" for r in range(n_rows)])
    filter_complex_rows = "; ".join([f"[{3*r}:v][{3*r+1}:v][{3*r+2}:v]hstack=inputs=3[t{r}]" for r in range(n_rows)])
    vstack_cmd = "".join([f"[t{r}]" for r in range(n_rows)]) + f"vstack=inputs={n_rows}[v]"
    
    # Only combine if all the test files are produced correctly.    
    if len(files) >= 5:
        subprocess.run(shlex.split(f"""ffmpeg -y \
            {inclusion_rows} \
            -filter_complex \
            \"{filter_complex_rows}; {vstack_cmd}\" \
            -map "[v]" \
            {output_video_name}
        """),
            # stdout = subprocess.STDOUT,
            # stderr = subprocess.STDOUT
        )

In [27]:
combine_test_videos("combined.mp4", ["anim1.mp4", "anim2.mp4", "anim3.mp4", "anim4.mp4", "anim4.mp4"])

Combining into ['anim1.mp4', 'anim2.mp4', 'anim3.mp4', 'anim4.mp4', 'anim4.mp4']


ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e